# Libs

In [4]:
from psw import MyClass # access information
import openai # openai lib
import tiktoken # lib used by openai to tokenize words
import pandas as pd

# Main Variables

In [5]:
access = MyClass()
openai.api_key = access.key # your key
model = access.model['model']['name'] # or 'gpt-3.5-turbo-0613'
input_cost = access.model['model']['input'] / 1000 # OpenIA charges by 1k tokens, dividing by 1000, will bring the cost by token
output_cost = access.model['model']['output'] / 1000 # same thing

In [6]:
print(f'{input_cost:.7f}')

0.0000015


In [7]:
print(f'{output_cost:.7f}')

0.0000020


# Functions

In [1]:
def input_text():
    text = input("user: ")
    return text

In [2]:
text = input_text()
text_2 = [{'content': text}]
print(text)
print(text_2)

whats the scape velocity of mars orbit?
[{'content': 'whats the scape velocity of mars orbit?'}]


In [9]:
# this function was copied from OpenAI Github account: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, model=model):
    text = [{'content': messages}] # modification made by me to deal with the inputed text
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
            "gpt-3.5-turbo-0613",
            "gpt-3.5-turbo-16k-0613",
            "gpt-4-0314",
            "gpt-4-32k-0314",
            "gpt-4-0613",
            "gpt-4-32k-0613",
            }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in text:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [30]:
encoding = tiktoken.encoding_for_model(model) # the encoder for our model 'gpt-3.5-turbo-0613'

In [46]:
text_example = text # extract the text
n_tokens_example = len(encoding.encode(text)) # count the number of tokens

print(f'The text: \n{text_example} \ncontains {n_tokens_example} tokens')


The text: 
whats the scape velocity of mars orbit? 
contains 9 tokens


In [48]:
text

'whats the scape velocity of mars orbit?'

In [40]:
tokens_per_message = 3 # constant
tokens_per_name = 1 # constant

num_tokens = 0 # token counter

for message in text: # couts the number of messages inside the 
    num_tokens += tokens_per_message # add 3 tokens to the number of tokens (considering the model 'gpt-3.5-turbo-0613')
    for key, value in message.items(): # key is the content value is the text inside content
        num_tokens += len(encoding.encode(value)) # adds the number of tokens encoded to the total number of tokens
        if key == "name":
            num_tokens += tokens_per_name # if 'name' is in key, it adds 1
num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
num_tokens

15

In [10]:
num_tokens_from_messages(text, model)

15

In [14]:
print(f'{15 * input_cost:.8f}')

0.00002250


## Explanation
Our text contains 9 tokens,
And 3 more tokens are added because of the model
and 3 more tokens are added because of the reply

Total: 15

The cost of this query will be 15 * input_cost

In [11]:
total_example = 15 * input_cost
print(f'Total Input Cost: {total_example:.8f}')

Total Input Cost: 0.00002250


In [12]:
def chat(model, text, max_tokens=500):
  response = openai.ChatCompletion.create(
    model=model,
    messages=[{"role": "user", "content" : text}],
    max_tokens=max_tokens
  ) 
  return response

# API Response

In [18]:
r = chat(model, 'whats the scape velocity of mars orbit?')
r

<OpenAIObject chat.completion id=chatcmpl-7qVnZ8CEkYL9ZyPuAg0GPmC48GM9z at 0x167a6191620> JSON: {
  "id": "chatcmpl-7qVnZ8CEkYL9ZyPuAg0GPmC48GM9z",
  "object": "chat.completion",
  "created": 1692749645,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The escape velocity of Mars orbit is about 5.03 kilometers per second (km/s), or approximately 11,223 miles per hour (mph)."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 16,
    "completion_tokens": 31,
    "total_tokens": 47
  }
}

In [18]:
in_cost = []
out_cost = []

In [19]:
def chatbot():
    while True:
        text = input_text()
        if text == 'EXIT':
            break
        else:
            pass
        
        print(f'Input Text: {text}')
        input_text_cost = num_tokens_from_messages(text, model) * input_cost
        print(f'Input Cost: {input_text_cost:.8f}')
        in_cost.append(input_text_cost)
        print('-' * 30)
        r = chat(model, text, max_tokens=100)
        
        output_text = r['choices'][0]['message']['content'].strip()
        print(f'Output: {output_text}')
        output_text_cost = r['usage']['completion_tokens'] * output_cost
        out_cost.append(output_text_cost)
        print(f'Output_cost: {output_text_cost:.8f}')
        print('-' * 30)
        print(f'Total cost of this query: {output_text_cost + input_text_cost:.8f}')
        print(f'Total cost of this Runtime: {sum(in_cost) + sum(out_cost):.8f}')
        print('-' * 30)

In [20]:
chatbot()

Input Text: whats the scape velocity of mars orbit?
Input Cost: 0.00002250
------------------------------
Output: The escape velocity of Mars' orbit is approximately 5.03 kilometers per second (11,186 miles per hour).
Output_cost: 0.00004800
------------------------------
Total cost of this query: 7.05e-05
Total cost of this Runtime: 7.05e-05
------------------------------


In [10]:
output_cost * 5000

0.01

In [25]:
input_cost * 6667

0.0100005

In [21]:
final_cost = sum(in_cost) + sum(out_cost)

In [22]:
print(f'{final_cost:.8f}')

0.00007050
